In [5]:

#BFS
import numpy as np
from collections import deque

class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

class Puzzle:
    def __init__(self, start, goal):
        self.start = (tuple(map(tuple, start)), self.find_empty(start))  # Dynamically find empty space
        self.goal = (tuple(map(tuple, goal)), self.find_empty(goal))
        self.solution = None
        self.num_explored = 0
        self.total_states_generated = 0  # Counter for total states generated

    def find_empty(self, state):
        # Find the position of the empty space (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)

    def neighbors(self, state):
        mat, (row, col) = state
        results = []
        directions = [(1, 0, 'down'), (-1, 0, 'up'), (0, 1, 'right'), (0, -1, 'left')]

        for dr, dc, action in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:  # Corrected condition
                mat1 = np.copy(mat)
                # Swap the empty space (0) with the adjacent number
                mat1[row][col], mat1[new_row][new_col] = mat1[new_row][new_col], mat1[row][col]
                results.append((action, (tuple(map(tuple, mat1)), (new_row, new_col))))
        return results

    def solve(self):
        start_node = Node(state=self.start, parent=None, action=None)
        queue = deque([start_node])  # Use deque for efficient pops from the left
        explored = set()  # Use a set for explored states

        while queue:
            node = queue.popleft()  # Dequeue the first node
            self.num_explored += 1

            # Check if the current node's state is the goal
            if node.state[0] == self.goal[0]:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return  # Found a solution

            # Mark the state as explored
            explored.add(node.state[0])

            # Explore neighbors
            for action, state in self.neighbors(node.state):
                if state[0] not in explored and all(node.state[0] != state[0] for node in queue):
                    child = Node(state=state, parent=node, action=action)
                    queue.append(child)  # Enqueue the child node
                    self.total_states_generated += 1  # Increment the total states generated counter

    def print_solution(self):
        if self.solution is None:
            print("No solution found.")
            return

        print("Start State:\n", np.array(self.start[0]), "\n")
        print("Goal State:\n", np.array(self.goal[0]), "\n")
        print("\nStates Explored: ", self.num_explored)
        print("Total States Generated: ", self.total_states_generated, "\n")

        print("Actions Taken to Reach the Goal:\n")
        for action, cell in zip(self.solution[0], self.solution[1]):
            print("Action: ", action)
            print(np.array(cell[0]), "\n")
        print("Goal Reached!!")

# Example usage
start = np.array([[1, 2, 3], [0, 4, 6], [7, 5, 8]])
goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

p = Puzzle(start, goal)
p.solve()
p.print_solution()

Start State:
 [[1 2 3]
 [0 4 6]
 [7 5 8]] 

Goal State:
 [[1 2 3]
 [4 5 6]
 [7 8 0]] 


States Explored:  14
Total States Generated:  26 

Actions Taken to Reach the Goal:

Action:  right
[[1 2 3]
 [4 0 6]
 [7 5 8]] 

Action:  down
[[1 2 3]
 [4 5 6]
 [7 0 8]] 

Action:  right
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

Goal Reached!!


In [9]:

#DFS
class Puzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.rows = 3
        self.cols = 3

    def get_neighbors(self, state):
        # Find the position of the blank (0)
        zero_pos = [(i, j) for i in range(self.rows) for j in range(self.cols) if state[i][j] == 0][0]
        x, y = zero_pos

        # Possible directions to move the blank space: up, down, left, right
        directions = [(-1, 0, 'up'), (1, 0, 'down'), (0, -1, 'left'), (0, 1, 'right')]
        neighbors = []

        for dx, dy, action in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.rows and 0 <= new_y < self.cols:
                new_state = [list(row) for row in state]  # Create a copy of the state
                # Swap blank with the neighboring tile
                new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
                neighbors.append((new_state, action))

        return neighbors

    def dfs(self):
        # Stack stores the current state, the path to the state, and the actions taken
        stack = [(self.initial_state, [], [])]  # (state, path, actions)
        visited = set()

        while stack:
            current_state, path, actions = stack.pop()

            # If we reached the goal, return the solution
            if current_state == self.goal_state:
                return path + [current_state], actions

            # Mark the current state as visited
            state_tuple = tuple(tuple(row) for row in current_state)
            if state_tuple not in visited:
                visited.add(state_tuple)

                # Explore all neighboring states
                for neighbor, action in self.get_neighbors(current_state):
                    stack.append((neighbor, path + [current_state], actions + [action]))

        return None, None  # If no solution found

    def print_solution(self, solution, actions):
        if solution:
            print("Solution found!")
            for step, action in zip(solution, actions + ['Goal Reached!!']):
                for row in step:
                    print(row)
                print(f"Action: {action}\n")
        else:
            print("No solution exists.")

# Example usage
initial_state = [
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

puzzle = Puzzle(initial_state, goal_state)
solution, actions = puzzle.dfs()
puzzle.print_solution(solution, actions)

Solution found!
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]
Action: right

[1, 2, 3]
[4, 6, 0]
[7, 5, 8]
Action: down

[1, 2, 3]
[4, 6, 8]
[7, 5, 0]
Action: left

[1, 2, 3]
[4, 6, 8]
[7, 0, 5]
Action: left

[1, 2, 3]
[4, 6, 8]
[0, 7, 5]
Action: up

[1, 2, 3]
[0, 6, 8]
[4, 7, 5]
Action: right

[1, 2, 3]
[6, 0, 8]
[4, 7, 5]
Action: right

[1, 2, 3]
[6, 8, 0]
[4, 7, 5]
Action: down

[1, 2, 3]
[6, 8, 5]
[4, 7, 0]
Action: left

[1, 2, 3]
[6, 8, 5]
[4, 0, 7]
Action: left

[1, 2, 3]
[6, 8, 5]
[0, 4, 7]
Action: up

[1, 2, 3]
[0, 8, 5]
[6, 4, 7]
Action: right

[1, 2, 3]
[8, 0, 5]
[6, 4, 7]
Action: right

[1, 2, 3]
[8, 5, 0]
[6, 4, 7]
Action: down

[1, 2, 3]
[8, 5, 7]
[6, 4, 0]
Action: left

[1, 2, 3]
[8, 5, 7]
[6, 0, 4]
Action: left

[1, 2, 3]
[8, 5, 7]
[0, 6, 4]
Action: up

[1, 2, 3]
[0, 5, 7]
[8, 6, 4]
Action: right

[1, 2, 3]
[5, 0, 7]
[8, 6, 4]
Action: right

[1, 2, 3]
[5, 7, 0]
[8, 6, 4]
Action: down

[1, 2, 3]
[5, 7, 4]
[8, 6, 0]
Action: left

[1, 2, 3]
[5, 7, 4]
[8, 0, 6]
Action: left

[1, 2, 3]
[5, 7